In [1]:
!pip3 install autokeras

     |████████████████████████████████| 166 kB 7.4 MB/s 
     |████████████████████████████████| 97 kB 8.8 MB/s 
     |████████████████████████████████| 454.4 MB 24 kB/s 
     |████████████████████████████████| 1.2 MB 59.0 MB/s 
     |████████████████████████████████| 4.0 MB 45.1 MB/s 
     |████████████████████████████████| 462 kB 77.3 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=a4b684eecf0de59e3085146d4eff47dd65a1d23bf6753d8233f68e771b72c9bb
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.43.0
    Uninstalling grpcio-1.43.0:
      Successfully uninstalled grpcio-1.43.0
  A

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from numpy import mean
from numpy import std
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat
import numpy as np
import PIL
import cv2
import os
from sklearn.model_selection import train_test_split
import autokeras as ak

In [139]:
x = list()
data = list()
z = 0
##Class-1 images##
folder_path_class1 = ('/content/drive/MyDrive/heart_sound_results_eval_new/STFT_Kaiser_512/AS/')
folder_path_class2 = ('/content/drive/MyDrive/heart_sound_results_eval_new/STFT_Kaiser_512/MR/')
folder_path_class3 = ('/content/drive/MyDrive/heart_sound_results_eval_new/STFT_Kaiser_512/MS/')
folder_path_class4 = ('/content/drive/MyDrive/heart_sound_results_eval_new/STFT_Kaiser_512/MVP/')
folder_path_class5 = ('/content/drive/MyDrive/heart_sound_results_eval_new/STFT_Kaiser_512/Normal/')
#folder_path_class1b = ('/content/drive/MyDrive/PCG_signal_time_frequency_image/Physionet_complete/scheme1/normal_TDPCT_SetA/')
#folder_path_class2b = ('/content/drive/MyDrive/PCG_signal_time_frequency_image/Physionet_complete/scheme1/normal_TDPCT_SetB/')
#folder_path_class3b = ('/content/drive/MyDrive/PCG_signal_time_frequency_image/Physionet_complete/scheme1/normal_TDPCT_SetC/')
#folder_path_class4b = ('/content/drive/MyDrive/PCG_signal_time_frequency_image/Physionet_complete/scheme1/normal_TDPCT_SetD/')
#folder_path_class5b = ('/content/drive/MyDrive/PCG_signal_time_frequency_image/Physionet_complete/scheme1/normal_TDPCT_SetF/')

class_types = {'MVP':0,'MR':1,'MS':2,'normal':3,'AS':4}

paths = [folder_path_class1,folder_path_class2,folder_path_class3,folder_path_class4,folder_path_class5]
#         folder_path_class1b,folder_path_class2b,folder_path_class3b,folder_path_class4b,folder_path_class5b,]
for p in paths:
  #print(p)
  for image in os.walk(p):
    data.append(image[2])
    #print(image[2])

  for i in range(len(data[0])):
    str_complete = p + data[0][i]
    #print(str_complete)
    img = cv2.imread(str_complete)
    img = cv2.resize(img, (128, 128))
    x.append(img)#Ensure all images are loaded
  data = []


In [140]:
y = np.ones([1000])
y[:200] =4
y[200:400]=1
y[400:600]=2
y[600:800]=0
y[800:1000]=3

In [141]:
data_x = np.asarray(x)
data_x.shape

(1000, 128, 128, 3)

In [142]:
y=y

In [189]:
x_train, x_test, y_train, y_test = train_test_split(data_x, y, test_size=0.1, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],5))
for i in range(np.array(y_train).shape[0]):
  label = y_train[i]
  y_tr_one_hot[i][int(label)] = 1

y_te_one_hot  = np.zeros((np.array(y_test).shape[0],5))
for i in range(np.array(y_test).shape[0]):
  label = y_test[i]
  y_te_one_hot[i][int(label)] = 1

y_val_one_hot  = np.zeros((np.array(y_val).shape[0],5))
for i in range(np.array(y_val).shape[0]):
  label = y_val[i]
  y_val_one_hot[i][int(label)] = 1


In [190]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ConvBlock(max_pooling=True)(output_node1)
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [191]:
print(x_train.shape)
print(y_tr_one_hot.shape)

# Feed the AutoModel with training data.
auto_model.fit(x_train, y_tr_one_hot, epochs=10)
# Predict with the best model.
predicted_y = auto_model.predict(x_test)

Trial 1 Complete [00h 00m 09s]
val_loss: 5.7307209968566895

Best val_loss So Far: 5.7307209968566895
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
25/25 [==============================] - 1s 28ms/step - loss: 5.5653 - accuracy: 0.4219
Epoch 2/10
25/25 [==============================] - 1s 28ms/step - loss: 2.3579 - accuracy: 0.7217
Epoch 3/10
25/25 [==============================] - 1s 27ms/step - loss: 1.5796 - accuracy: 0.7637
Epoch 4/10
25/25 [==============================] - 1s 28ms/step - loss: 1.0929 - accuracy: 0.8132
Epoch 5/10
25/25 [==============================] - 1s 27ms/step - loss: 0.8462 - accuracy: 0.8539
Epoch 6/10
25/25 [==============================] - 1s 27ms/step - loss: 0.7219 - accuracy: 0.8742
Epoch 7/10
25/25 [==============================] - 1s 28ms/step - loss: 0.8176 - accuracy: 0.8551
Epoch 8/10
25/25 [==============================] - 1s 28ms/step - loss: 0.8784 - accuracy: 0.8386
Epoch 9/10
25/25 [==================

In [192]:
predicted_y = auto_model.predict(x_test)

4/4 [==============================] - 0s 14ms/step


In [193]:
y_pred = np.argmax(predicted_y, axis=1)

In [194]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred)
print("confusion matrix \n",cm1)

confusion matrix 
 [[ 8  6  4  2  1]
 [ 0 13  1  1  0]
 [ 2  2 18  2  2]
 [ 0  2  6  9  1]
 [ 1  0  0  0 19]]


In [195]:
precision = sklearn.metrics.precision_score(y_test,y_pred,average='micro')
print(precision)

0.67


In [196]:
accuracy=np.diag(cm1).sum()/cm1.sum().sum()
print(accuracy)

0.67


In [197]:
recall = sklearn.metrics.recall_score(y_test,y_pred,average='micro')
print(recall)

0.67


In [198]:
K_cappa = sklearn.metrics.cohen_kappa_score(y_test,y_pred)
print(K_cappa)

0.5853229454636844
